In [1]:
import numpy as np
import pandas as pd
import re
from pyvi import ViTokenizer 
import string

## Load data

In [2]:
train = pd.read_excel(r'Data\data_lazada.xlsx')

In [3]:
train.head()

,Sentence,Label
0,Đã nhận hàng nhanh chóng! Nhưng Side XXL mờ Sh...,0
1,giao hàng lâu. áo ko giống mẫu chụp. thất vọng,0
2,"Áo đẹp trong tầm giá, mỗi tội mình mua 2 cái đ...",1
3,áo đẹp lắm ạ,1
4,"chuẩn y hình, đẹp ưng lắm luôn,cho shop 5sao",1


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  1171 non-null   object
 1   Label     1171 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 18.4+ KB


## Count Positive(1) and Negative (0)

In [5]:
count_sentiment = pd.value_counts(train['Label'])
count_sentiment

1    666
0    505
Name: Label, dtype: int64

## DataProcessing

In [6]:
def pre_processing(text):
    
    # chuyển về chữ thường
    text = text.lower()
    
    # Xóa icon
    emoji_pattern = re.compile("["
                   u"\U0001F600-\U0001F64F"  # emoticons
                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                   u"\U00002702-\U000027B0"
                   u"\U000024C2-\U0001F251"
                   u"\U0001f926-\U0001f937"
                   u"\u200d"
                   u"\u2640-\u2642" 
                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Chuan hoa tieng viet, tieng anh, thuat ngu
    replace_list={
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
    'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
    'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
    'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
    'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
    #Chuẩn hóa 1 số sentiment words/English words
    ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
    'okey': ' ok ', 'ôkê': ' ok ', ' oki ': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
    ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ', 'cam on':u'cám ơn',u'cảm ơn':'cám ơn',
    ' not ': u' không ',' khoong ': ' không ', u' kg ': u' không ','ko ': 'không', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' k ':u' không ',u'chẳng':u'không',u'đéo':u'không',
    'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
    ' lol ': ' negative ',' cc ': ' negative ','cute': u' dễ thương ','huhu': ' negative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', ' j ': u' gì ', '“': ' ',
    ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
    'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
    'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
    'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
    'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
    'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
    'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
    'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
    'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
    'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ', ' quả ng ': ' quảng ',
    'dep': u' đẹp ',u'xầu':u'xấu',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u' qủa': u' quả', 
    ' iu ': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ', 
    ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
    'qc':u'quảng cáo',u' éo ':' negative ',' bik ':' biết ',
    #dưới 3* quy về 1*, trên 3* quy về 5*
    '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ', '5 sao': '5star',
    'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ', '3 sao':'5star','4 sao':'5star','1 sao':'1star',
    '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ','1*':' 1star ', '2*':' 1star ','3*':' 5star ','4*':' 5star ','5*':' 5star '
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)

    #Xóa dấu câu linh tinh
    text = re.sub('['+string.punctuation+']', ' ', text)

    # Tokenizer
    text = ViTokenizer.tokenize(text)
    
    _not = ['không', 'không thể' , 'chẳng', 'đéo', 'đếch', 'kém']
    positive = ['5star', 'positive', 'ưng', 'hài_lòng', 'thích',  'ngon', 'tốt']
    negative = ['1start', 'negative', 'lừa','thất_vọng', 'chán', 'hợp', 'xấu', 'lừa_đảo', 'rất kém','tệ', 'tồi_tệ']
    txt_split = [txt.replace('_',' ') for txt in text.split()]
    for i in range(len(txt_split)):
        if txt_split[i] in _not:
            if (i < len(txt_split)-1):
                if txt_split[i+1] in positive:
                    text = text.replace(txt_split[i]+' '+txt_split[i+1],'negative')
                    txt_split[i]=txt_split[i+1]=''
                elif txt_split[i+1] in negative:
                    text = text.replace(txt_split[i]+' '+txt_split[i+1],'positive')
                    txt_split[i]=txt_split[i+1]=''
        else:
            if txt_split[i] in negative:
                text = text + ' negative'
            elif txt_split[i] in positive:
                text = text + ' positive'
    return text

In [7]:
New_Sentence = []
for i in range(train.shape[0]):
    New_Sentence.append(pre_processing(train['Sentence'][i]))
train['New_Sentence'] = New_Sentence

In [8]:
train.head()

,Sentence,Label,New_Sentence
0,Đã nhận hàng nhanh chóng! Nhưng Side XXL mờ Sh...,0,đã nhận hàng nhanh_chóng nhưng side xxl mờ cửa...
1,giao hàng lâu. áo ko giống mẫu chụp. thất vọng,0,giao hàng lâu áo khônggiống mẫu chụp thất_vọng
2,"Áo đẹp trong tầm giá, mỗi tội mình mua 2 cái đ...",1,áo đẹp trong tầm giá mỗi tội mình mua 2 cái đề...
3,áo đẹp lắm ạ,1,áo đẹp lắm ạ
4,"chuẩn y hình, đẹp ưng lắm luôn,cho shop 5sao",1,chuẩn_y hình đẹp ưng lắm luôn cho cửa_hàng 5st...


## Split data

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train.loc[:,'New_Sentence'], train.loc[:,'Label'], test_size =.2)

## Support Vector Machine

In [11]:
from sklearn.svm import SVC 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [12]:
stopwords=('rằng', 'thì', 'mà', 'là', 'thế', 'à', 'ừ', 'vậy', 'như')
pipe_line = Pipeline([
    ("vectorizer", CountVectorizer(stop_words=stopwords)), # bag of words
    ("tfidf", TfidfTransformer()), # tf-idf
    ("clf_svm", SVC(kernel='linear', probability=True)) # svm kernel = 'linear'
])

In [13]:
clf_svm = pipe_line.fit(X_train, y_train)

In [14]:
print("Train accuracy:",clf_svm.score(X_train, y_train))
print("Test accuracy:", clf_svm.score(X_test, y_test))

Train accuracy: 0.9700854700854701
Test accuracy: 0.9148936170212766


In [15]:
from sklearn.metrics import classification_report
y_predict = clf_svm.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       109
           1       0.93      0.91      0.92       126

    accuracy                           0.91       235
   macro avg       0.91      0.92      0.91       235
weighted avg       0.92      0.91      0.91       235

